In [1]:
import numpy as np
import cv2
import os
import imutils

In [2]:
NMS_Threshold = 0.3
MIN_Confidence = 0.2

In [3]:
def pedestrian_detection(image, model, layer_name, personidz=0):
    (H, W) = image.shape[:2]
    results = []

    
    blb = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    model.setInput(blb)
    layerOutputs = model.forward(layer_name)


    boxes = []
    centroids = []
    confidences = []

    for result in layerOutputs:
        for detection in result:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if classID == personidz and confidence > MIN_Confidence:
                box = detection[0:4]*np.array([W,H,W,H])
                (centerX , centerY, width, height) = box.astype("int")
                x = int(centerX - (width/2))
                y = int(centerY - (height/2))
                boxes.append([x,y,int(width),int(height)])
                centroids.append((centerX,centerY))
                confidences.append(float(confidence))

        idzs = cv2.dnn.NMSBoxes(boxes,confidences,MIN_Confidence,NMS_Threshold)
        if len(idzs)>0:
            for i in idzs.flatten():
                (x,y) = (boxes[i][0] ,boxes[i][1])
                (w,h) = (boxes[i][2] , boxes[i][3])
                res = (confidences[i],(x,y,x+w,y+h),centroids[i])
                results.append(res)
        return results

In [4]:
labelspth = "coco.names"
Label  = open(labelspth).read().strip().split("\n")
weights_path = "yolov4-tiny.weights"
config_path = "yolov4-tiny.cfg"

model = cv2.dnn.readNetFromDarknet(config_path,weights_path)

'''
model.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
model.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

'''
layername = model.getLayerNames()
layername = [layername[i-1] for i in model.getUnconnectedOutLayers()]
capture = cv2.VideoCapture("General_public_preview.mp4")
writer = None


In [5]:
while True:
    (grabbed,image) = capture.read()
    if not grabbed:
        break
    image = imutils.resize(image,width = 700)
    results = pedestrian_detection(image,model,layername,personidz = Label.index("person"))
    for res in results:
        cv2.rectangle(image,(res[1][0],res[1][1]),(res[1][2],res[1][3]),(0,255,0),2)
    cv2.imshow("DETECTION",image)
    key = cv2.waitKey(1)
    if key == 27:
        break


In [6]:

capture. Release()
cv2.destroyAllWindows()